Imports

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Classifiers
from sklearn.ensemble import RandomForestClassifier

# Common imports
import numpy as np
import pandas as pd
import os
import tarfile
import cv2
import pickle

Set numpy's random-state to 42 to make this notebook's output stable across runs.

In [ ]:
np.random.seed(42)

**Reload data** decides if a new dataframe has to be created. \
Default is False, but will be set to True if no dataframe exists yet.\
Can be set to True to force the program to overwrite an existing dataframe.


In [ ]:
RELOAD_DATA = False

Helper function 'target_value'.

In [ ]:
def target_value(val):
    if val == 'aanwezig':
        return 2
    if val == 'buiten':
        return 1
    return 0

Check if directory 'model' exists.

In [ ]:
if not os.path.isdir('./model/'):
    os.mkdir('./model/')

Check if a dataframe exists, and create a new one if this is not the case.
Else load the existing dataframe.

In [ ]:
if not os.path.isfile('./model/dataframe.sav'):
    RELOAD_DATA = True # When there is not yet a dataframe, create one

if RELOAD_DATA: # Check whether there needs to be created a new dataframe
    #Extract when not already extracted
    if not os.path.isdir('./data/classificatie'):
        if not os.path.isfile('./data/classificatie.tar'):
            raise Exception('classificatie.tar not fount')

        print('Extracting tar...')
        tar = tarfile.open('./data/classificatie.tar')
        tar.extractall('./data/')
        tar.close()
        print('Extracting tar Done!')

    if not os.path.isdir('./classificatie'):
        raise Exception('Extracted files not found')


    # Get grayscale values from pictures
    print('Creating dataframe')
    samples = []
    sample_counter = 0
    musti = pd.DataFrame()

    for folder in os.listdir('./classificatie/'):
        for file in os.listdir(f'./classificatie/{folder}'):
            img = cv2.imread(f'./classificatie/{folder}/{file}', 0)
            img = cv2.normalize(img,np.zeros((640, 352)), 0, 1000, cv2.NORM_MINMAX)
            # add them to a dataframe
            imgd = dict()
            imgd['target'] = target_value(folder)
            c = 0
            for i in img.flatten():
                c += 1
                imgd[f'p{c}'] = i
            samples.append(imgd)
            sample_counter+=1

            if sample_counter % 200==0:
                temp_df = pd.DataFrame.from_dict(samples)
                musti = musti.append(temp_df, ignore_index=True)
                samples = []
    temp_df = pd.DataFrame.from_dict(samples)
    musti = musti.append(temp_df, ignore_index=True)
    samples = []

    print('Saving DataFrame')
    
    pickle.dump(musti, open('./model/dataframe.sav', 'wb'))
else:
    print('Loading DataFrame')
    musti = pickle.load(open('./model/dataframe.sav', 'rb'))

print('DataFrame Loaded')
print(musti)

In [ ]:
smallest_dataset_len = min(len(musti[musti.target == 2]), len(musti[musti.target == 1]), len(musti[musti.target == 0]))
for i in range(3):
    frac = 1 - smallest_dataset_len / len(musti[musti.target == i])
    print(frac, i)
    musti = musti.drop(musti.query(f'target == {i}').sample(frac=frac).index)

print(len(musti[musti.target == 2]), len(musti[musti.target == 1]), len(musti[musti.target == 0]))

Create training and test data

In [ ]:
X, y = musti.drop('target', axis=1), musti['target']
y = y.astype(np.uint8)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Create and fit the chosen model

In [ ]:
model = RandomForestClassifier(n_estimators= 185, max_features= 11, random_state= 42)
print('fitting model')


model.fit(X_train, y_train)
pickle.dump(model, open('./model/model.sav', 'wb'))